# Visualizing Global Land Cover

* The Global Land Cover 30m product is a powerful dataset for global land cover change monitoring
* See https://gee-community-catalog.org/tutorials/examples/glc_fcs30d_lulc for source documentation



Copyright 2024 Ian Housman

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geeviz/blob/master/examples/global_land_cover_example_notebook.ipynb) 
[![github](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geeViz/blob/master/examples/global_land_cover_example_notebook.ipynb)


In [1]:
try:
    import  geeViz.geeView as geeView
except:
    !python -m pip install geeViz
    import  geeViz.geeView as geeView

ee = geeView.ee
Map = geeView.Map

print('done')

Initializing GEE
Cached project id file path: C:\Users\ihousman\.config\earthengine\credentials.proj_id
Cached project id: lcms-292214
Successfully initialized
geeViz package folder: c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\geeViz
done


## Make a basic viewer

* This is a slight adaptation of the example provided here: https://gee-community-catalog.org/tutorials/examples/glc_fcs30d_lulc/#steps
* geeViz will automatically generate legends and query lookups using `"autoViz" : True` in the visualization params
* Double-clicking on the map will show the values for the pixel you clicked on

In [4]:
Map.clearMap()
Map.port = 1234
# Specify projection to use for zonal summaries and map querying
# Be sure to leave one of scale or transform as None
crs= 'EPSG:4326'
transform = None
scale = 30


Map.setQueryCRS(crs)
if transform == None:
    Map.setQueryScale(scale)
else:
    Map.setQueryTransform(transform)
Map.clearMapLayers()

####################################################################
# Adapted from: https://gee-community-catalog.org/tutorials/examples/glc_fcs30d_lulc/#steps
annual = ee.ImageCollection('projects/sat-io/open-datasets/GLC-FCS30D/annual')



# Five-Yearly data for 1985-90, 1990-95 and 1995-2000
fiveyear = ee.ImageCollection('projects/sat-io/open-datasets/GLC-FCS30D/five-years-map')

# Classification scheme has 36 classes (35 landcover classes and 1 fill value)
classValues = [10, 11, 12, 20, 51, 52, 61, 62, 71, 72, 81, 82, 91, 92, 120, 121, 122, 130, 140, 150, 152, 153, 181, 182, 183, 184, 185, 186, 187, 190, 200, 201, 202, 210, 220, 0]
classNames = ['Rainfed_cropland', 'Herbaceous_cover_cropland', 'Tree_or_shrub_cover_cropland', 'Irrigated_cropland', 'Open_evergreen_broadleaved_forest', 'Closed_evergreen_broadleaved_forest', 'Open_deciduous_broadleaved_forest', 'Closed_deciduous_broadleaved_forest', 'Open_evergreen_needle_leaved_forest', 'Closed_evergreen_needle_leaved_forest', 'Open_deciduous_needle_leaved_forest', 'Closed_deciduous_needle_leaved_forest', 'Open_mixed_leaf_forest', 'Closed_mixed_leaf_forest', 'Shrubland', 'Evergreen_shrubland', 'Deciduous_shrubland', 'Grassland', 'Lichens_and_mosses', 'Sparse_vegetation', 'Sparse_shrubland', 'Sparse_herbaceous', 'Swamp', 'Marsh', 'Flooded_flat', 'Saline', 'Mangrove', 'Salt_marsh', 'Tidal_flat', 'Impervious_surfaces', 'Bare_areas', 'Consolidated_bare_areas', 'Unconsolidated_bare_areas', 'Water_body', 'Permanent_ice_and_snow', 'Filled_value']
classColors = ['ffff64', 'ffff64', 'ffff00', 'aaf0f0', '4c7300', '006400', 'a8c800', '00a000', '005000', '003c00', '286400', '285000', 'a0b432', '788200', '966400', '964b00', '966400', 'ffb432', 'ffdcd2', 'ffebaf', 'ffd278', 'ffebaf', '00a884', '73ffdf', '9ebb3b', '828282', 'f57ab6', '66cdab', '444f89', 'c31400', 'fff5d7', 'dcdcdc', 'fff5d7', '0046c8', 'ffffff', 'ffffff']

# Mosaic the data into a single image
annualMosaic = annual.mosaic()
fiveYearMosaic = fiveyear.mosaic()

# Rename bands from b1, b2, etc. to 2000, 2001, etc.
fiveYearsList = ee.List.sequence(1985, 1995, 5).map(lambda year:ee.Number(year).format('%04d') )
fiveyearMosaicRenamed = fiveYearMosaic.rename(fiveYearsList)
yearsList = ee.List.sequence(2000, 2022).map(lambda year: ee.Number(year).format('%04d'))
annualMosaicRenamed = annualMosaic.rename(yearsList)
years = fiveYearsList.cat(yearsList).getInfo()

# Convert the multiband image to an ImageCollection
def getFiveYear(year):
    date = ee.Date.fromYMD(ee.Number.parse(year), 1, 1)
    return fiveyearMosaicRenamed.select([year]).set({'system:time_start': date.millis(), 'system:index': year, 'year': ee.Number.parse(year)})
fiveYearlyMosaics = fiveYearsList.map(getFiveYear)

def getAnnualYear(year):
    date = ee.Date.fromYMD(ee.Number.parse(year), 1, 1)
    return annualMosaicRenamed.select([year]).set({'system:time_start': date.millis(), 'system:index': year, 'year': ee.Number.parse(year)})
yearlyMosaics = yearsList.map(getAnnualYear)

allMosaics = fiveYearlyMosaics.cat(yearlyMosaics)
mosaicsCol = ee.ImageCollection.fromImages(allMosaics)

# Set up viz properties
out_band_name = "lulc"


# Function to convert a given band into a time-enabled image object
def setupLulc(img):
    img = img.set(
        {
            f"{out_band_name}_class_names": classNames,
            f"{out_band_name}_class_palette": classColors,
            f"{out_band_name}_class_values": classValues,
        }
    )
    return img

# Rename band and set the properties for visualization
landcoverCol = mosaicsCol.select([0],[out_band_name]).map(setupLulc)

print('Pre-processed Collection', landcoverCol.getInfo())


# Visualize the most common class
Map.addLayer(landcoverCol, {'reducer':ee.Reducer.mode(),'autoViz':True}, 'Global Land Cover' )

# Querying the data by double-clicking on the map will show the full time series of values
Map.turnOnInspector()
Map.view()

Setting click query crs to: EPSG:4326
Setting click query scale to: 30
Pre-processed Collection {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'lulc', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'lulc_class_values': [10, 11, 12, 20, 51, 52, 61, 62, 71, 72, 81, 82, 91, 92, 120, 121, 122, 130, 140, 150, 152, 153, 181, 182, 183, 184, 185, 186, 187, 190, 200, 201, 202, 210, 220, 0], 'system:time_start': 473385600000, 'lulc_class_names': ['Rainfed_cropland', 'Herbaceous_cover_cropland', 'Tree_or_shrub_cover_cropland', 'Irrigated_cropland', 'Open_evergreen_broadleaved_forest', 'Closed_evergreen_broadleaved_forest', 'Open_deciduous_broadleaved_forest', 'Closed_deciduous_broadleaved_forest', 'Open_evergreen_needle_leaved_forest', 'Closed_evergreen_needle_leaved_forest', 'Open_deciduous_needle_leaved_forest', 'Closed_deciduous_needle_leaved_forest

## Calculating and Exporting Landcover Statistics¶

* geeViz streamlines calculating and exporting statistics 
* Once the map loads, you can use the `TOOLS` under `Area Tools` to generate charts
* Charts can be downloaded as a PNG and the underlying data can be downloaded as a CSV

* This example shows the imageCollection as a time lapse
* Once the map loads, you can annimate the time series using the time lapse controls

In [11]:
Map.clearMapLayers()
Map.turnOffLayersWhenTimeLapseIsOn = False # Set this to False to avoid layers automatically turning off when a time lapse is turned on

Map.addTimeLapse(landcoverCol, {'autoViz':True,'years':years,'canAreaChart':True,'areaChartParams':{'crs':crs,'scale':scale,'transform':transform}}, 'Global Land Cover' )

# Add country boundaries as a layer to select with
countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
Map.addSelectLayer(countries, {}, "Global Country Boundaries")

Map.turnOnAutoAreaCharting()
Map.view()

Adding layer: Global Land Cover
Adding layer: Global Country Boundaries
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd a:\GEE\gee_py_modules_package\geeViz\examples
geeView URL: http://localhost:8001/geeView/?projectID=lcms-292214&accessToken=ya29.a0AcM612wkg8MbxbrDXNW8mteX4Rd499FHQEVJdbOZj1H-GwUzSY62J5l3FmqkFSkPTRf5Tk4AucXiwJkjKsKgpmaZDQkTfF0N0DBGIoAGs4YaBQNZi4QGs89ILI67zLR7H3bnN-xxYTOTbbhUSGAtAd6ttI41X6ayCqBAdsDK1oEaCgYKAXISARESFQHGX2MiIDqlQ3V3yMgYr_x_W4UIug0178


## Create Sankey Charts

* You can also create Sankey charts and download transition matrices with geeViz
* Once the map loads, you can use the `TOOLS` under `Area Tools` to generate charts
* You can customize the years to include in the sankey charts under the `Area Tools Parameters` -> `Transition Charting Periods`
* Charts can be downloaded as a PNG and the underlying data can be downloaded as a CSV

In [20]:
Map.clearMapLayers()

Map.addTimeLapse(landcoverCol, {'autoViz':True,'years':years,'canAreaChart':True,'areaChartParams':{'line':True,'sankey':True,'crs':crs,'transform':transform,'scale':scale}}, 'Global Land Cover' )

countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
Map.addSelectLayer(countries, {}, "Global Country Boundaries")

Map.turnOnAutoAreaCharting()
Map.view()

Adding layer: Global Land Cover
Adding layer: Global Country Boundaries
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd a:\GEE\gee_py_modules_package\geeViz\examples
geeView URL: http://localhost:8001/geeView/?projectID=lcms-292214&accessToken=ya29.a0AcM612zmF5WYJo9uMvskGFuWThtweN3c0z3oPOZMcP6Q2dYif-XM9e919oU5_4xr325sbggaBfy17yplckLIfw_slYInMx9QZBcNsnM7QTI1PUfzAKCTFHjgfxrKc0UYcDAcQ0qp3SdgpIt9bRJtRU6iLzIGTY68LXsYXY4jtH4aCgYKAdoSARESFQHGX2Mi3ccUX5TpycA17Q1JKCZzhA0178
